In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.6/663.6 kB 25.3 MB/s eta 0:00:00


### Import packages

Import the necessary packages.

In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
# Used to securely store your API key
from google.colab import userdata

In [5]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [7]:
model = genai.GenerativeModel('gemini-pro')

In [8]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00


In [9]:
from rdflib import Graph

def load_ontology(ontology_url):
    # Create a new graph
    g = Graph()

    # Parse the ontology from the given URL
    g.parse(ontology_url, format='turtle')

    return g

g = load_ontology("/content/Cartoon.ttl")

In simple cases, the `response.text` accessor is all you need. To display formatted Markdown text, use the `to_markdown` function:

##Task 1

In [14]:
%%time
intro = "Our project aims to develop a system that validates ontologies using Language Models (LMs) and publicly available Knowledge Graphs (KGs)"
queries = ["Ontology Hierarchy Validation: Generate all the subclass- superclass structures, and ensure the subclass-superclass relationships are correctly structured"
            # ,"Property Domain/Range Validation: Validate that the assigned domain and range of properties are suitable based on class definitions and instances."
           ]

q = intro +"Given this ontology in a turtle file " +  g.serialize(format="turtle") +  "Perform this task and give appropriate solution: " + queries[0]
response1 = model.generate_content(q)

print("Superclass Subclass Heirarchy Identified within the ontology")
display(to_markdown(response1.text))
prompt = "For every superclass, generate a list of possible subclasses in the following response: "+response1.text +". If there are n super classes, then there must be n lists for sub classes."
response2 = model.generate_content(prompt)

print("List of possible subclasses for each superclass identified by the LLM")
display(to_markdown(response2.text))
# responses1.append(response.text)
prompt2 =  "Validate whether the subclasses identified in" + response2.text + "  include the following subclasses: " + response1.text +". Enlist the validated subclasses as well."
response3 = model.generate_content(prompt2)

print("Validated subclasses: ")
display(to_markdown(response3.text))



Superclass Subclass Heirarchy Identified within the ontology


> **Subclass-Superclass Structures:**
> 
> * **http://dbpedia.org/ontology/Anime**
>     * http://dbpedia.org/ontology/Cartoon
>     * http://dbpedia.org/ontology/Comic
>     * http://dbpedia.org/ontology/Manga
>     * http://dbpedia.org/ontology/Work
>     * http://dbpedia.org/ontology/WrittenWork
> * **http://dbpedia.org/ontology/Cartoon**
>     * http://dbpedia.org/ontology/TelevisionShow
>     * http://dbpedia.org/ontology/Film
> * **http://dbpedia.org/ontology/Work**
>     * http://schema.org/CreativeWork
> 
> **Validation of Subclass-Superclass Relationships:**
> 
> All the subclass-superclass relationships in the provided ontology appear to be correctly structured. Here's a breakdown:
> 
> * **http://dbpedia.org/ontology/Anime** is a subclass of **http://dbpedia.org/ontology/Work**, which is consistent with the definition of anime as a type of work.
> * **http://dbpedia.org/ontology/Cartoon** is a subclass of both **http://dbpedia.org/ontology/Anime** and **http://dbpedia.org/ontology/Work**, which accurately reflects the nature of cartoons as animated works.
> * **http://dbpedia.org/ontology/TelevisionShow** is a subclass of **http://dbpedia.org/ontology/Cartoon**, which is correct as television shows can be animated.
> * **http://dbpedia.org/ontology/Film** is a subclass of **http://dbpedia.org/ontology/Cartoon**, which is also accurate as animated films are a type of cartoon.
> * **http://schema.org/CreativeWork** is a superclass of **http://dbpedia.org/ontology/Work**, which aligns with the broader category of creative works that includes anime, cartoons, and other types of works.
> 
> Therefore, the subclass-superclass relationships in the given ontology are valid and follow a logical hierarchy.

List of possible subclasses for each superclass identified by the LLM


> **Subclass-Superclass Lists:**
> 
> * **http://dbpedia.org/ontology/Anime**:
>     * Cartoon
>     * Comic
>     * Manga
>     * Work
>     * WrittenWork
> * **http://dbpedia.org/ontology/Cartoon**:
>     * TelevisionShow
>     * Film
> * **http://dbpedia.org/ontology/Work**:
>     * CreativeWork

Validated subclasses: 


> **Validated Subclasses:**
> 
> * **http://dbpedia.org/ontology/Anime**
>     * **http://dbpedia.org/ontology/Cartoon**
>     * **http://dbpedia.org/ontology/Comic**
>     * **http://dbpedia.org/ontology/Manga**
>     * **http://dbpedia.org/ontology/Work**
>     * **http://dbpedia.org/ontology/WrittenWork**
> * **http://dbpedia.org/ontology/Cartoon**
>     * **http://dbpedia.org/ontology/TelevisionShow**
>     * **http://dbpedia.org/ontology/Film**
> * **http://dbpedia.org/ontology/Work**
>     * **http://schema.org/CreativeWork**

CPU times: user 268 ms, sys: 30.3 ms, total: 299 ms
Wall time: 17.2 s


##Task 2

In [ ]:
%%time
intro = "Our project aims to develop a system that validates ontologies using Language Models (LMs) and publicly available Knowledge Graphs (KGs)"
queries = ["Property Domain/Range Validation: Identify all the relationships and properties in this ttl file, with their domain and range"]

q = intro +"Given this ontology in a turtle file " +  g.serialize(format="turtle") +  "Perform this task and give appropriate solution: " + queries[0]
response1 = model.generate_content(q)
display(to_markdown(response1.text))

print("Properties Identified within the ontology")
display(to_markdown(response1.text))
prompt = "For every property identified, generate a list of possible domain and range in the following response: "+response1.text +". If there are n properties, then there must be n lists for domain and range."
response2 = model.generate_content(prompt)

print("List of possible domain and range for the properties")
display(to_markdown(response2.text))
# responses1.append(response.text)
prompt2 =  "Validate whether the domain and range identified in" + response2.text + "  include the following domain and range: " + response1.text +". Enlist the validated properties as well."
response3 = model.generate_content(prompt2)

print("Validated properties: ")
display(to_markdown(response3.text))

##Fetching better ontologies

In [19]:
!pip install SPARQLWrapper

In [20]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, URIRef
import os

sparql = SPARQLWrapper("https://dbpedia.org/sparql")

# Fetch distinct classes from DBpedia
sparql.setQuery("""
    SELECT DISTINCT ?class WHERE {
      [] a ?class .
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Post-processing
ontology_classes = []
for result in results["results"]["bindings"]:
    class_uri = result["class"]["value"]
    if class_uri.startswith("http://dbpedia.org/ontology/"):
        ontology_classes.append(class_uri)

In [42]:
from urllib.parse import quote

sparql = SPARQLWrapper("https://dbpedia.org/sparql")

# Fetch distinct classes from DBpedia
sparql.setQuery("""
    SELECT DISTINCT ?class WHERE {
      [] a ?class .
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Post-processing
ontology_classes = []
for result in results["results"]["bindings"]:
    class_uri = result["class"]["value"]
    if class_uri.startswith("http://dbpedia.org/ontology/"):
        ontology_class_encoded = class_uri
        ontology_classes.append(ontology_class_encoded)


In [ ]:
ontology_classes

In [46]:
from urllib.parse import quote

# Define a function to fetch relationships/properties of a given class
def fetch_properties_for_class(class_uri):
    sparql.setQuery(f"""
        SELECT DISTINCT ?property ?domain ?range WHERE {{
          ?subject rdf:type <{class_uri}> ;
                   ?property [] .
          OPTIONAL {{
            ?property rdfs:domain ?domain .
            ?property rdfs:range ?range .
          }}
        }}
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    # Extract properties
    properties = [result["property"]["value"] for result in results["results"]["bindings"]]

    return properties

# Fetch properties for each ontology class
class_properties_mapping = {}
for ontology_class_uri in ontology_classes:
    decoded_class_uri = quote(ontology_class_uri, safe='/:')
    properties = fetch_properties_for_class(decoded_class_uri)
    class_properties_mapping[ontology_class_uri] = properties

# Print the mapping of classes to their properties
for ontology_class_uri, properties in class_properties_mapping.items():
    print(f"Class: {ontology_class_uri}")
    print("Properties:")
    for prop in properties:
        print(f"- {prop}")
    print()


Streaming output truncated to the last 5000 lines.
- http://dbpedia.org/property/silverMex
- http://dbpedia.org/property/umpire
- http://dbpedia.org/property/coachesalt
- http://dbpedia.org/property/constructorName
- http://dbpedia.org/property/medaltemplatesExpand
- http://dbpedia.org/property/mlbaffiliation
- http://dbpedia.org/property/olyMedals
- http://dbpedia.org/property/summerYouthOlympicsApps
- http://dbpedia.org/property/summerYouthOlympicsBest
- http://dbpedia.org/property/summerYouthOlympicsFirst
- http://dbpedia.org/property/wrestlingarena
- http://dbpedia.org/property/youthWorldCupApps
- http://dbpedia.org/property/youthWorldCupBest
- http://dbpedia.org/property/youthWorldCupFirst
- http://dbpedia.org/property/ground(s)_
- http://dbpedia.org/property/province
- http://dbpedia.org/property/coordinates
- http://dbpedia.org/property/county
- http://dbpedia.org/property/clubGaa
- http://dbpedia.org/property/exNames
- http://dbpedia.org/ontology/deathPlace
- http://dbpedia.org

In [45]:
import os

# Define a function to fetch relationships/properties of a given class and save them to a TTL file
def fetch_properties_and_save(class_uri):
    decoded_class_uri = quote(class_uri, safe='/:')
    properties = fetch_properties_for_class(decoded_class_uri)

    # Create a folder to store TTL files if it doesn't exist
    folder_name = "class_ttls"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Write properties to a TTL file
    file_name = f"{class_uri.split('/')[-1]}.ttl"
    file_path = os.path.join(folder_name, file_name)
    with open(file_path, 'w') as f:
        for prop in properties:
            f.write(f"{prop}\n")

# Fetch properties for each ontology class and save to TTL files
for ontology_class_uri in ontology_classes:
    fetch_properties_and_save(ontology_class_uri)


In [36]:
!rm -rf "/content/Ontologies"